In [1]:
from keras.preprocessing.image import ImageDataGenerator

from keras.applications import InceptionV3
from keras import models
from keras import layers
from keras import optimizers

from keras.callbacks import EarlyStopping

import numpy as np
import gc

"""
./format_data/train/
./format_data/val/
"""
train_dir = "./format_data/train/"
test_dir = "./format_data/val/"

feature_size = 299
batch_size = 5

call_back = EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    verbose=0, 
    mode='max'
)

Using TensorFlow backend.


In [2]:
train_gen = ImageDataGenerator(rescale=1./255,rotation_range = 360,fill_mode='reflect',
                               width_shift_range = 0.2,height_shift_range = 0.2,
                               shear_range = 0.2,
                               horizontal_flip=True,vertical_flip=True)
test_gen = ImageDataGenerator(rescale=1./255)

train_gener = train_gen.flow_from_directory(train_dir,target_size=(feature_size,feature_size),batch_size=batch_size,shuffle = True,seed = 100,class_mode="categorical")
test_gener = test_gen.flow_from_directory(test_dir,target_size=(feature_size,feature_size),batch_size=batch_size,class_mode="categorical")

for data_batch,label_batch in train_gener:
    print(data_batch.shape)
    print(label_batch.shape)
    break

Found 1884 images belonging to 12 classes.
Found 252 images belonging to 12 classes.
(5, 299, 299, 3)
(5, 12)


In [3]:
I_model = InceptionV3(weights=None,include_top=False,input_shape=(feature_size,feature_size,3))
I_model.load_weights('inception_v3_weights_notop.h5')
I_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 149, 149, 32)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 149, 149, 32)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [4]:
model = models.Sequential()

model.add(I_model)
model.add(layers.Flatten())
model.add(layers.Dense(1000,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(12,activation='softmax'))

KeyboardInterrupt: 

In [ ]:
model.summary()

In [6]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=1e-4),metrics=['acc'])
history = model.fit_generator(train_gener,steps_per_epoch=200,epochs=25,
                              validation_data=test_gener,validation_steps=25,callbacks=[call_back])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/25
200/200 [==============================] - 109s - loss: 7.4061 - acc: 0.5340 - val_loss: 14.6997 - val_acc: 0.0880
Epoch 2/25
200/200 [==============================] - 86s - loss: 7.6843 - acc: 0.5233 - val_loss: 14.9291 - val_acc: 0.0738
Epoch 3/25
200/200 [==============================] - 84s - loss: 7.5594 - acc: 0.5310 - val_loss: 14.8286 - val_acc: 0.0800
Epoch 4/25
200/200 [==============================] - 84s - loss: 7.5392 - acc: 0.5323 - val_loss: 14.7969 - val_acc: 0.0820
Epoch 5/25
200/200 [==============================] - 84s - loss: 7.7528 - acc: 0.5190 - val_loss: 14.6997 - val_acc: 0.0880
Epoch 6/25
200/200 [==============================] - 84s - loss: 7.6924 - acc: 0.5227 - val_loss: 14.9291 - val_acc: 0.0738
Epoch 7/25
 11/200 [>.............................] - ETA: 72s - loss: 6.4472 - acc: 0.6000

KeyboardInterrupt: 

In [ ]:
model.save('Inception_v3_whole.h5')